In [ ]:
import pandas as pd

# 讀取 CSV 文件
file_path = "/mnt/data/housing_data.csv"
df = pd.read_csv(file_path)

# 清理數據（從 row2 開始）
df_cleaned = df.iloc[1:].copy()

# 轉換數據類型並填補缺失值
numeric_columns = ["Area", "No. of Rooms", "No. of Bathrooms", 
                   "Miles (dist. between school and house)", 
                   "Rent Price per Month", "Sell Price"]
df_cleaned[numeric_columns] = df_cleaned[numeric_columns].apply(pd.to_numeric, errors='coerce').fillna(0)

# 計算租房 48 個月的總成本
df_cleaned["Total Rent (48 months)"] = df_cleaned["Rent Price per Month"] * 48

# **Q3: 找出離學校最近的租房**
nearest_rental = df_cleaned.loc[df_cleaned["Miles (dist. between school and house)"].idxmin()]
print("Q3: 離學校最近的租房")
print(nearest_rental)

# **Q4: 買房 vs 租房**
# 判斷租房 48 個月總成本是否低於房價
if nearest_rental["Total Rent (48 months)"] < nearest_rental["Sell Price"]:
    decision = "租房"
    best_option = nearest_rental
else:
    decision = "買房"
    best_option = df_cleaned.sort_values(by="Miles (dist. between school and house)").iloc[0]

# 輸出結果
print(f"\nQ4: 最佳選擇 -> {decision}")
print(best_option)

# **Q5: 租金或售價異常的房屋考慮面積與地點**
df_cleaned["Rent per Sq Unit"] = df_cleaned["Rent Price per Month"] / df_cleaned["Area"]
df_cleaned["Sell per Sq Unit"] = df_cleaned["Sell Price"] / df_cleaned["Area"]

def find_outliers_by_area(column):
    mean = df_cleaned[column].mean()
    std = df_cleaned[column].std()
    lower_bound = mean - 2 * std
    upper_bound = mean + 2 * std
    return df_cleaned[(df_cleaned[column] < lower_bound) | (df_cleaned[column] > upper_bound)]

rent_outliers = find_outliers_by_area("Rent per Sq Unit")
sell_outliers = find_outliers_by_area("Sell per Sq Unit")

# 按地點計算異常租金與房價
def find_outliers_by_location():
    outliers = []
    for location in df_cleaned["Location"].unique():
        subset = df_cleaned[df_cleaned["Location"] == location]
        for column in ["Rent Price per Month", "Sell Price"]:
            mean = subset[column].mean()
            std = subset[column].std()
            lower_bound = mean - 2 * std
            upper_bound = mean + 2 * std
            outliers.extend(subset[(subset[column] < lower_bound) | (subset[column] > upper_bound)].index.tolist())
    return df_cleaned.loc[outliers]

location_outliers = find_outliers_by_location()

print("\nQ5: 異常租金或房價的房屋（考慮面積與地點）")
print("異常租金（每平方單位）:")
print(rent_outliers)
print("\n異常房價（每平方單位）:")
print(sell_outliers)
print("\n按地點計算的異常租金與房價:")
print(location_outliers)
